## PCA Model for Image Reconstruction

In [5]:
# load the mnist dataset
import numpy as np
import os

file_path = "C:\\Users\\HP\\Desktop\\Image-Reconstruction\\Data\\mnist_preprocessed.npz"

In [6]:
data = np.load(file_path)
X_train = data["X_train"]
X_test = data["X_test"]

In [11]:
X_train.shape

(60000, 784)

In [12]:
X_test.shape

(10000, 4704)

### PCA Computation